#ETL PROJECT - Chiara Galbraith


EXTRACTION

Download cvs file from data.wa.gov.au to local folder (PROJECT_2)

Children in the CEO's care on 30 June 2008-2017
https://catalogue.data.wa.gov.au/dataset/cbbcac73-1948-4722-82df-50fcfedf1660/resource/0109e6d3-a77c-4ace-8be2-9c36267c4279/download/dpc-data.csv

Safety and wellbeing assessements undertaken where concerns for childern were investigated by the Department for Child Protection and Family Support during 2015-16 and 2016-17
https://catalogue.data.wa.gov.au/dataset/10183f4f-b656-4204-b6c0-b471aad68de7/resource/d47adbf5-78ab-4d9e-9498-5e48c26df88d/download/swa-dpc-data.csv

Create a Python file in VS Code import the two csv files

TRANSFORM 

CLEANING: Delete age group column from care_df because all the age ranges are 0-17 years therefore this column does not provide any useful information.

CLEANING: Delete gender from care_df as the gender is not disclosed therefore the column is not relevant.

FILTERING: The data for assessments was only for the periods 2015-16 and 2016-17 whereas the children in care data began in 2008 and went beyond 2017. 

For the care data to be able to be compared with the assessment data the years needed to be compatible. To acheive this all data for the years prior to 2015 were removed (dropped) and all the data after the year 2016 were removed (dropped).

CLEANING: Then repeated the deletion of gender and age from the care data as per the assessed data.

CLEANING: Because the assessment data read as 2016-16 and 2016-17, again for ease of comparison only the first section of the date was kept. 2015-16 became 2015 by removing the "-" then dropping the last 2 digits "-2". This was repeated for the 2016-17 year to 
become 2017.

To ensure compatibility, the ACC Regions in both data sets were compared. These matched.

To enable a merge of the 2 data bases an index was added to the care df.

MERGE: The two dataframes were merged but to differentiate between the care data columns and the assessment data columns a suffix of care or assessed was added to column names. eg Count_care and Count_assessed.

A new data frame was created that only included the columns required in the preferred order. This was because both data sets had the same headings therefore it was the most efficient way to remove "duplicates" eg AACC Regions_assessed and AACC Regions_care. In the end the only two data columns from both data sets included were "Count_assessed" and "Count_care"

CLEANING: All column headings were renamed.

LOADING
A new database ETL_DB was created in PGAdmin.

A table called children_at_risk was created with in the database with the headings that correspond to the final python data frame.

In python a connection was created with the data base in postgres.

Then the final data frame in python was connected the the table created in the postres with data from the data frame being populated into the table.


ISSUES encountered:

EXTRACTING: The csv files that were originally downloaded must have become corrupted as they would not load in VScode via jupyter or python. The error kept reading file does not exist. The csb files were moved to different locations and on the face of it there was no way to know that there were issues with actual file. When the files were deleted and then downloaded again the process was simple.

LOADING: Issues included postgres only appears to read lowercase letters in the headers and does not like spaces. First moving spaces and changing the headings in postgres and python did not work, it wasnt until the table schema was open and the columns wiht in there expanded was it obvious that the reason python and postgres were not connecting was because in Postgres it converted all capitals in the header to lower case. Again this was not obvious and therefore a lot of time was lost trying to problem solve this process.
 









In [1]:
import pandas as pd     
from sqlalchemy import create_engine

In [2]:
#(E)xtract
#reading in CSV file
care_df = pd.read_csv("care.csv")
care_df.head()

,_id,Indicator,AACC Region,Aboriginality,Age Group,Gender,Count,Year
0,1,Children in care,East Kimberley,NonAboriginal,0 - 17 Years,All,0,2008
1,2,Children in care,East Kimberley,Aboriginal,0 - 17 Years,All,91,2008
2,3,Children in care,Goldfields-Esperance,NonAboriginal,0 - 17 Years,All,34,2008
3,4,Children in care,Goldfields-Esperance,Aboriginal,0 - 17 Years,All,69,2008
4,5,Children in care,Midwest-Gascoyne,Aboriginal,0 - 17 Years,All,94,2008


In [3]:
#(E)xtract
#reading in CSV file
assessed_df = pd.read_csv("assessed.csv")
assessed_df.head()

,_id,Indicator,AACC Region,Aboriginality,Age Group,Gender,Count,Year
0,1,Safety and Wellbeing Assessments,East Kimberley,Aboriginal,0 - 17 Years,All,325,2015-16
1,2,Safety and Wellbeing Assessments,East Kimberley,NonAboriginal,0 - 17 Years,All,4,2015-16
2,3,Safety and Wellbeing Assessments,Goldfields-Esperance,Aboriginal,0 - 17 Years,All,193,2015-16
3,4,Safety and Wellbeing Assessments,Goldfields-Esperance,NonAboriginal,0 - 17 Years,All,126,2015-16
4,5,Safety and Wellbeing Assessments,Midwest-Gascoyne,Aboriginal,0 - 17 Years,All,471,2015-16


In [4]:
#(T)ransform
# Delete age group column (not required)
del care_df['Age Group']
care_df.head()

,_id,Indicator,AACC Region,Aboriginality,Gender,Count,Year
0,1,Children in care,East Kimberley,NonAboriginal,All,0,2008
1,2,Children in care,East Kimberley,Aboriginal,All,91,2008
2,3,Children in care,Goldfields-Esperance,NonAboriginal,All,34,2008
3,4,Children in care,Goldfields-Esperance,Aboriginal,All,69,2008
4,5,Children in care,Midwest-Gascoyne,Aboriginal,All,94,2008


In [5]:
#(T)ransform
#Delete gender (not relevant)
del care_df['Gender']
care_df.head()

,_id,Indicator,AACC Region,Aboriginality,Count,Year
0,1,Children in care,East Kimberley,NonAboriginal,0,2008
1,2,Children in care,East Kimberley,Aboriginal,91,2008
2,3,Children in care,Goldfields-Esperance,NonAboriginal,34,2008
3,4,Children in care,Goldfields-Esperance,Aboriginal,69,2008
4,5,Children in care,Midwest-Gascoyne,Aboriginal,94,2008


In [6]:
#(T)ransform
#delete years prior to 2015
care2_df = care_df.loc[care_df['Year']>2014]
care2_df.head()

,_id,Indicator,AACC Region,Aboriginality,Count,Year
126,127,Children in care,East Kimberley,Aboriginal,144,2015
127,128,Children in care,East Kimberley,NonAboriginal,0,2015
128,129,Children in care,Goldfields-Esperance,NonAboriginal,23,2015
129,130,Children in care,Goldfields-Esperance,Aboriginal,135,2015
130,131,Children in care,Midwest-Gascoyne,Aboriginal,142,2015


In [7]:
#(T)ransform
#Delete years after 2016
care3_df = care2_df.loc[care2_df['Year']<2017]
care3_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 126 to 161
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   _id            36 non-null     int64 
 1   Indicator      36 non-null     object
 2   AACC Region    36 non-null     object
 3   Aboriginality  36 non-null     object
 4   Count          36 non-null     int64 
 5   Year           36 non-null     int64 
dtypes: int64(3), object(3)
memory usage: 2.0+ KB


In [8]:
#(T)ransform
care4_df=care3_df
care4_df['id1']=range(36)
care4_df.head()

,_id,Indicator,AACC Region,Aboriginality,Count,Year,id1
126,127,Children in care,East Kimberley,Aboriginal,144,2015,0
127,128,Children in care,East Kimberley,NonAboriginal,0,2015,1
128,129,Children in care,Goldfields-Esperance,NonAboriginal,23,2015,2
129,130,Children in care,Goldfields-Esperance,Aboriginal,135,2015,3
130,131,Children in care,Midwest-Gascoyne,Aboriginal,142,2015,4


In [9]:
#(T)ransform
# # Delete age group column (not required)
del assessed_df['Age Group']


In [10]:
#(T)ransform
#Delete gender (not relevant)
del assessed_df['Gender']


In [11]:
#(T)ransform
#Remove dash in year by replacing with no space
assessed_df['Year'] = assessed_df['Year'].str.replace(r'-','')


In [12]:
#assessed_df.head()

In [13]:
#(T)ransform
#Remove last 2 digits to make single year (do not re run as it removes entire year)
assessed_df['Year'] = assessed_df['Year'].str[:-2]

In [14]:
assessed_df.head()

,_id,Indicator,AACC Region,Aboriginality,Count,Year
0,1,Safety and Wellbeing Assessments,East Kimberley,Aboriginal,325,2015
1,2,Safety and Wellbeing Assessments,East Kimberley,NonAboriginal,4,2015
2,3,Safety and Wellbeing Assessments,Goldfields-Esperance,Aboriginal,193,2015
3,4,Safety and Wellbeing Assessments,Goldfields-Esperance,NonAboriginal,126,2015
4,5,Safety and Wellbeing Assessments,Midwest-Gascoyne,Aboriginal,471,2015


In [15]:
#Check regions in assessed data
assessed_df['AACC Region'].unique()


array(['East Kimberley', 'Goldfields-Esperance', 'Midwest-Gascoyne',
       'Perth', 'Pilbara', 'Southern', 'Western Australia', 'Wheatbelt',
       'West Kimberley'], dtype=object)

In [16]:
#check regions in care data
care4_df['AACC Region'].unique()

array(['East Kimberley', 'Goldfields-Esperance', 'Midwest-Gascoyne',
       'Perth', 'Pilbara', 'Southern', 'Western Australia', 'Wheatbelt',
       'West Kimberley'], dtype=object)

In [17]:
#(T)ransform
#add index to care df to enable merge
care5_df=care4_df.set_index('id1')

In [18]:
#(T)ransform
#merge df's and add suffixes to identify original df
merged_df = care5_df.merge(assessed_df,left_index=True, right_index=True,suffixes =('_care','_assessed'),validate=None)
merged_df.head()

,_id_care,Indicator_care,AACC Region_care,Aboriginality_care,Count_care,Year_care,_id_assessed,Indicator_assessed,AACC Region_assessed,Aboriginality_assessed,Count_assessed,Year_assessed
id1,,,,,,,,,,,,
0,127,Children in care,East Kimberley,Aboriginal,144,2015,1,Safety and Wellbeing Assessments,East Kimberley,Aboriginal,325,2015
1,128,Children in care,East Kimberley,NonAboriginal,0,2015,2,Safety and Wellbeing Assessments,East Kimberley,NonAboriginal,4,2015
2,129,Children in care,Goldfields-Esperance,NonAboriginal,23,2015,3,Safety and Wellbeing Assessments,Goldfields-Esperance,Aboriginal,193,2015
3,130,Children in care,Goldfields-Esperance,Aboriginal,135,2015,4,Safety and Wellbeing Assessments,Goldfields-Esperance,NonAboriginal,126,2015
4,131,Children in care,Midwest-Gascoyne,Aboriginal,142,2015,5,Safety and Wellbeing Assessments,Midwest-Gascoyne,Aboriginal,471,2015


In [19]:
merged_df.columns

Index(['_id_care', 'Indicator_care', 'AACC Region_care', 'Aboriginality_care',
       'Count_care', 'Year_care', '_id_assessed', 'Indicator_assessed',
       'AACC Region_assessed', 'Aboriginality_assessed', 'Count_assessed',
       'Year_assessed'],
      dtype='object')

In [20]:
#(T)ransform
#create new data frame with only columns required in order
organise_df = merged_df[["Year_care","AACC Region_care", "Aboriginality_care", "Count_assessed", "Count_care"]]
organise_df.head()

,Year_care,AACC Region_care,Aboriginality_care,Count_assessed,Count_care
id1,,,,,
0,2015,East Kimberley,Aboriginal,325,144
1,2015,East Kimberley,NonAboriginal,4,0
2,2015,Goldfields-Esperance,NonAboriginal,193,23
3,2015,Goldfields-Esperance,Aboriginal,126,135
4,2015,Midwest-Gascoyne,Aboriginal,471,142


In [53]:
#(T)ransform
#rename columns
Final_df = organise_df.rename(columns={"AACC Region_care":"aacc_region", "Aboriginality_care":"aboriginality","Year_care":"years", "Count_assessed":"number_assessed", "Count_care":"number_in_care"})
Final_df.head()



,years,aacc_region,aboriginality,number_assessed,number_in_care
id1,,,,,
0,2015,East Kimberley,Aboriginal,325,144
1,2015,East Kimberley,NonAboriginal,4,0
2,2015,Goldfields-Esperance,NonAboriginal,193,23
3,2015,Goldfields-Esperance,Aboriginal,126,135
4,2015,Midwest-Gascoyne,Aboriginal,471,142


In [54]:
Final_df.columns

Index(['years', 'aacc_region', 'aboriginality', 'number_assessed',
       'number_in_care'],
      dtype='object')

In [55]:
#(L)oading
#Export file as csv, without pandas index, but with headers
Final_df.to_csv("ChildrenData.csv", index=False, header=True)

In [56]:
Final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   years            36 non-null     int64 
 1   aacc_region      36 non-null     object
 2   aboriginality    36 non-null     object
 3   number_assessed  36 non-null     int64 
 4   number_in_care   36 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 2.9+ KB


In [60]:
#(L)oading
rds_connection_string="postgres:XXXXXXX@localhost:5432/ETL_DB"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [61]:
engine.table_names()

['children_at_risk']

In [62]:
#(L)oading
Final_df.to_sql(name = 'children_at_risk', con=engine, if_exists='append', index=True)